In [1]:
import h5py
from model_xray.config import *
from model_xray.path_manager import pm

with h5py.File(pm.get_mcwa_path('famous_le_10m'), 'r') as f:
    for mz_name in f:
        print(mz_name)
        mz = f[mz_name]
        print(mz.dtype, mz.shape)

DenseNet121
float32 (1, 8062504)
EfficientNetV2B0
float32 (1, 7200312)
EfficientNetV2B1
float32 (1, 8212124)
MobileNet
float32 (1, 4253864)
MobileNetV2
float32 (1, 3538984)
MobileNetV3Large
float32 (1, 5507432)
MobileNetV3Small
float32 (1, 2554968)
NASNetMobile
float32 (1, 5326716)


In [3]:
from model_xray.config_classes import XLSBAttackConfig, PayloadType
from model_xray.utils.mal_embedding_utils import x_lsb_attack, x_lsb_extract

import numpy as np

arr = np.zeros((10, ), dtype=np.uint8)

payload = np.array([0, 1, 2, 3, 4, 5, 6, 7, 20, 14], dtype=np.uint8)
payload_bytes = payload.tobytes()

config = XLSBAttackConfig(
    x = 3,
    fill=True,
    payload_type=PayloadType.PYTHON_BYTES,
    payload_bytes=payload_bytes
)

arr_attacked = x_lsb_attack(arr, config)
print(arr)
print(arr_attacked)

extracted_bytes = x_lsb_extract(arr_attacked, config)

payload_bytes_as_array = np.frombuffer(payload_bytes, dtype=np.uint8)
extracted_bytes_as_array = np.frombuffer(extracted_bytes, dtype=np.uint8)

print(payload_bytes_as_array)
print(extracted_bytes_as_array)

assert extracted_bytes == payload_bytes

capacity: 30, n_b: 8
payload bits: 00000000000000010000001000000011000001000000010100000110000001110001010000001110, len(bits): 80
dupe_amount: 1
len(bits): 80
reshaped: Array('bin3', ['000', '000', '000', '000', '000', '100', '000', '010', '000', '000']), remainder: Array('bin50', ['11000001000000010100000110000001110001010000001110'])
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 4 0 2 0 0]
capacity: 30, n_b: 8
bits: Array('bin3', ['000', '000', '000', '000', '000', '100', '000', '010', '000', '000'])
[ 0  1  2  3  4  5  6  7 20 14]
[0 1 2 0]


AssertionError: 

In [1]:
import math
import numpy as np
from bitstring import Dtype
import itertools
import os

import traceback


from model_xray.config_classes import XLSBAttackConfig, PayloadType
from model_xray.utils.mal_embedding_utils import _x_lsb_attack_numpy, _x_lsb_attack_bitstring, _x_lsb_attack_numpy_bin, x_lsb_extract, mcwa_to_bytes_arr, bytes_arr_to_mcwa

def check_attack(arr, config, attack_func):
    x = config.x

    config.msb = False
    arr_attacked = attack_func(arr, config)

    config.msb = True

    n_b = arr.dtype.itemsize * 8
    unattacked_bits_amnt = n_b - x

    config.x = unattacked_bits_amnt

    unattacked_bytes_orig = x_lsb_extract(arr, config)
    unattacked_bytes_attacked = x_lsb_extract(arr_attacked, config)

    if not unattacked_bytes_orig == unattacked_bytes_attacked:
        print("orig:", mcwa_to_bytes_arr(arr))
        print("attacked:", mcwa_to_bytes_arr(arr_attacked))

        print("extracted orig:", np.frombuffer(unattacked_bytes_orig, dtype=np.uint8))
        print("extracted attacked:", np.frombuffer(unattacked_bytes_attacked, dtype=np.uint8))

    assert unattacked_bytes_orig == unattacked_bytes_attacked

    config.msb = False
    config.x = x
    extracted_bytes = x_lsb_extract(arr_attacked, config)

    assert extracted_bytes == config.payload_bytes

    return arr_attacked

rng = np.random.default_rng()

dtypes = [np.uint16, np.uint32, np.uint64, np.float32, np.float64]

n_ws = [8, 16, 32, 64, 1_000_000]

config = XLSBAttackConfig(
    x = 3,
    fill=True,
    payload_type=PayloadType.PYTHON_BYTES,
    payload_bytes=None
)

tests_c = 0
try:
    for dtype, n_w in itertools.product(dtypes, n_ws):
        dtype_str = dtype.__name__
        # print(dtype_str)
        n_bits = Dtype(dtype_str).length
        # print(n_bits)

        for x in range(1, n_bits-1):
        # for x in [4, 6, 8,]:
            capacity = x*n_w
            byte_capacity = math.ceil(capacity / 8)

            randbytes = os.urandom(byte_capacity)

            config.x = x
            config.payload_bytes = randbytes

            if np.issubdtype(dtype, np.floating):
                arr = rng.random(size=(n_w, ), dtype=dtype)
            elif np.issubdtype(dtype, np.integer):
                arr = rng.integers(low=0, high = 2**n_bits, size=(n_w,), dtype=dtype)
            else:
                raise ValueError(f"Unknown dtype {dtype}")

            # print(f"dtype: {dtype}, n_w: {n_w}, x: {x}")
            arr_attacked_bitstring = check_attack(arr, config, _x_lsb_attack_numpy_bin)

            if x % 8 == 0:
                arr_attacked_numpy = check_attack(arr, config, _x_lsb_attack_numpy)
                if not np.array_equal(arr_attacked_numpy, arr_attacked_bitstring):
                    # print(f"dtype: {dtype}, n_w: {n_w}, x: {x}")
                    # print("orig: \n", mcwa_to_bytes_arr(arr))
                    # print("numpy: \n", mcwa_to_bytes_arr(arr_attacked_numpy))
                    # print("bitstring: \n", mcwa_to_bytes_arr(arr_attacked_bitstring))
                    pass
                assert np.array_equal(arr_attacked_numpy, arr_attacked_bitstring)
            
            tests_c += 1
except Exception as e:
    print(f"dtype: {dtype}, n_w: {n_w}, x: {x}")
    print(e)
    print(traceback.format_exc())
    pass

print(f"Tests passed: {tests_c}") 


Tests passed: 990


In [3]:
import time
import numpy as np

rng = np.random.default_rng()

n_w = 100_000_000
arr = rng.random(size=(n_w, ), dtype=np.float32)

config = XLSBAttackConfig(
    x = 8,
    fill=True,
    payload_type=PayloadType.RANDOM,
    payload_bytes=None
)

start = time.time()
arr_attacked = _x_lsb_attack_numpy(arr, config)
print("bytes ver took: ", time.time() - start)

start = time.time()
arr_attacked = _x_lsb_attack_numpy_bin(arr, config)
print("bin ver took: ", time.time() - start)

bytes ver took:  1.6777446269989014
bin ver took:  3.7816061973571777


In [3]:
import math
import numpy as np
from bitstring import Dtype
import itertools
import os

from model_xray.config_classes import XLSBAttackConfig, PayloadType
from model_xray.utils.mal_embedding_utils import _x_lsb_attack_numpy, _x_lsb_attack_bitstring, _x_lsb_attack_numpy_bin, x_lsb_extract, mcwa_to_bytes_arr, bytes_arr_to_mcwa

arr = np.array([0b11111110, 0b11111100, 0b11111000], dtype=np.uint8)

payload = np.array([255,], dtype=np.uint8).tobytes()

config = XLSBAttackConfig(
    x = 2,
    fill=True,
    payload_type=PayloadType.PYTHON_BYTES,
    payload_bytes=payload
)

arr_attacked = _x_lsb_attack_numpy_bin(arr, config)

def ret_binstr_arr(a: np.ndarray):
    return [np.binary_repr(x, 8) for x in a]

print(ret_binstr_arr(arr))
print(ret_binstr_arr(arr_attacked))

['11111110', '11111100', '11111000']
['11111111', '11111111', '11111011']


In [3]:
import numpy as np
from bitstring import BitArray, Array

from model_xray.config_classes import XLSBAttackConfig, PayloadType
from model_xray.utils.mal_embedding_utils import _x_lsb_attack_numpy, _x_lsb_attack_bitstring, x_lsb_extract, mcwa_to_bytes_arr, bytes_arr_to_mcwa

n_w = 8
n_bits = 16
dtype = np.uint16

rng = np.random.default_rng()
# arr = rng.integers(low=0, high = 2**n_bits, size=(n_w,), dtype=dtype)

arr = np.array([1, 2, 3, 4, 5, 6, 7, 8], dtype=dtype)

c = Array(str(arr.dtype).lower())
c.fromfile(arr.tobytes())

print(c)

# recon = np.frombuffer(c.tobytes(), dtype=dtype).reshape(arr.shape)

# print("orig: \n", mcwa_to_bytes_arr(arr))
# print("recon: \n", mcwa_to_bytes_arr(recon))

Array('uint16', [256, 512, 768, 1024, 1280, 1536, 1792, 2048])


In [3]:
import numpy as np

n_b = 8
byte_amnt = n_b // 8
dt = np.dtype(f'uint{n_b}')

arr = np.array([1, 2, 3, 4, 5, 6, 7, 8], dtype=np.uint8)
unpacked = np.unpackbits(arr, bitorder='big').reshape(len(arr), byte_amnt, n_b)
print(unpacked)

[[[0 0 0 0 0 0 0 1]]

 [[0 0 0 0 0 0 1 0]]

 [[0 0 0 0 0 0 1 1]]

 [[0 0 0 0 0 1 0 0]]

 [[0 0 0 0 0 1 0 1]]

 [[0 0 0 0 0 1 1 0]]

 [[0 0 0 0 0 1 1 1]]

 [[0 0 0 0 1 0 0 0]]]


In [24]:
import numpy as np

# n_b = 8
# byte_amnt = n_b // 8
# dt = np.dtype(f'uint{n_b}')

# count = 4

arr = np.array([[1, 2], [100, 200]], dtype=np.uint8)
# arr.resize((len(arr), 1))
arr.resize((*arr.shape,))
unpacked = np.unpackbits(arr, axis=-1, count=6, bitorder='big')
print(unpacked)


addition = np.ones(shape=(2,2), dtype=np.uint8)

print(addition)
stacked = np.hstack([unpacked, addition])
print(stacked)
final = np.packbits(stacked, axis=-1, bitorder='big')

print(final)

[[0 0 0 0 0 0]
 [0 1 1 0 0 1]]
[[1 1]
 [1 1]]
[[0 0 0 0 0 0 1 1]
 [0 1 1 0 0 1 1 1]]
[[  3]
 [103]]
